In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import numpy as np
import pandas as pd
import nltk
import re
from nltk.corpus import stopwords
from nltk.tokenize import sent_tokenize,word_tokenize
from gensim.models import Word2Vec
from keras.layers import Embedding, LSTM, Dense, Dropout, Lambda, Flatten
from keras.models import Sequential, load_model
# Instead of importing model_from_config, you should directly load your model using load_model
# from keras.models import Sequential, load_model, model_from_config
import keras.backend as K
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.metrics import cohen_kappa_score

In [5]:
df = pd.read_csv("/content/drive/MyDrive/training_set_rel3.tsv", sep='\t', encoding='ISO-8859-1');
df.dropna(axis=1,inplace=True)
df.drop(columns=['domain1_score','rater1_domain1','rater2_domain1'],inplace=True,axis=1)
df.head()
temp = pd.read_csv("/content/drive/MyDrive/Processed_data.csv")
temp.drop("Unnamed: 0",inplace=True,axis=1)

In [6]:
df['domain1_score']=temp['final_score']
df.head()

,essay_id,essay_set,essay,domain1_score
0,1,1,"Dear local newspaper, I think effects computer...",6
1,2,1,"Dear @CAPS1 @CAPS2, I believe that using compu...",7
2,3,1,"Dear, @CAPS1 @CAPS2 @CAPS3 More and more peopl...",5
3,4,1,"Dear Local Newspaper, @CAPS1 I have found that...",8
4,5,1,"Dear @LOCATION1, I know having computers has a...",6


In [7]:
df['essay'][0]

"Dear local newspaper, I think effects computers have on people are great learning skills/affects because they give us time to chat with friends/new people, helps us learn about the globe(astronomy) and keeps us out of troble! Thing about! Dont you think so? How would you feel if your teenager is always on the phone with friends! Do you ever time to chat with your friends or buisness partner about things. Well now - there's a new way to chat the computer, theirs plenty of sites on the internet to do so: @ORGANIZATION1, @ORGANIZATION2, @CAPS1, facebook, myspace ect. Just think now while your setting up meeting with your boss on the computer, your teenager is having fun on the phone not rushing to get off cause you want to use it. How did you learn about other countrys/states outside of yours? Well I have by computer/internet, it's a new way to learn about what going on in our time! You might think your child spends a lot of time on the computer, but ask them so question about the econom

In [8]:
temp.head(1)

,essay_id,essay_set,essay,final_score,clean_essay,char_count,word_count,sent_count,avg_word_len,spell_err_count,noun_count,adj_count,verb_count,adv_count
0,1,1,"Dear local newspaper, I think effects computer...",6,Dear local newspaper I think effects computer...,1441,344,16,4.188953,11,76,75,18,24


In [9]:
#Make Dataset
y = df['domain1_score']
df.drop('domain1_score',inplace=True,axis=1)
X=df

In [10]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [11]:
X_train.shape

(9083, 3)

In [12]:
train_e = X_train['essay'].tolist()
test_e = X_test['essay'].tolist()

In [16]:
import nltk
import re
from nltk.corpus import stopwords

# Download the 'stopwords' dataset
nltk.download('stopwords')

# Download the 'punkt' dataset for sentence tokenization
nltk.download('punkt')  # This line is added to download the missing resource

train_sents=[]
test_sents=[]

# Now you can access the stopwords
stop_words = set(stopwords.words('english'))

def sent2word(x):
    x=re.sub("[^A-Za-z]"," ",x)
    x.lower()
    filtered_sentence = []
    words=x.split()
    for w in words:
        if w not in stop_words:
            filtered_sentence.append(w)
    return filtered_sentence

def essay2word(essay):
    essay = essay.strip()
    tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')
    raw = tokenizer.tokenize(essay)
    final_words=[]
    for i in raw:
        if(len(i)>0):
            final_words.append(sent2word(i))
    return final_words

for i in train_e:
    train_sents+=essay2word(i)

for i in test_e:
    test_sents+=essay2word(i)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [17]:
len(train_sents)


116500

In [18]:
train_sents[0]

['It',
 'first',
 'day',
 'high',
 'school',
 'gut',
 'full',
 'butterflies',
 'make',
 'want',
 'run',
 'bathrooms',
 'hide',
 'world']

In [19]:
def get_model():
    model = Sequential()
    model.add(LSTM(300, dropout=0.4, recurrent_dropout=0.4, input_shape=[1, 300], return_sequences=True))
    model.add(LSTM(64, recurrent_dropout=0.4))
    model.add(Dropout(0.5))
    model.add(Dense(1, activation='relu'))
    model.compile(loss='mean_squared_error', optimizer='rmsprop', metrics=['mae'])
    model.summary()
    return model

In [21]:
#Training Word2Vec model
num_features = 300
min_word_count = 40
num_workers = 4
context = 10
downsampling = 1e-3

# Install gensim if you haven't already
!pip install gensim

from gensim.models import Word2Vec

# Change 'size' to 'vector_size'
model = Word2Vec(train_sents,
                 workers=num_workers,
                 vector_size=num_features,  # Changed 'size' to 'vector_size'
                 min_count=min_word_count,
                 window=context,
                 sample=downsampling)

model.init_sims(replace=True)
model.wv.save_word2vec_format('word2vecmodel.bin', binary=True)

<ipython-input-21-a8daf568ad59>:21: DeprecationWarning: Call to deprecated `init_sims` (Gensim 4.0.0 implemented internal optimizations that make calls to init_sims() unnecessary. init_sims() is now obsoleted and will be completely removed in future versions. See https://github.com/RaRe-Technologies/gensim/wiki/Migrating-from-Gensim-3.x-to-4).
  model.init_sims(replace=True)


In [23]:
def makeVec(words, model, num_features):
    vec = np.zeros((num_features,),dtype="float32")
    noOfWords = 0.
    # Use index_to_key instead of index2word
    index2word_set = set(model.wv.index_to_key)
    for i in words:
        if i in index2word_set:
            noOfWords += 1
            vec = np.add(vec,model.wv[i]) # Use model.wv[i] to access word vectors
    vec = np.divide(vec,noOfWords)
    return vec


def getVecs(essays, model, num_features):
    c=0
    essay_vecs = np.zeros((len(essays),num_features),dtype="float32")
    for i in essays:
        essay_vecs[c] = makeVec(i, model, num_features)
        c+=1
    return essay_vecs


clean_train=[]
for i in train_e:
    clean_train.append(sent2word(i))
training_vectors = getVecs(clean_train, model, num_features)

clean_test=[]

for i in test_e:
    clean_test.append(sent2word(i))
testing_vectors = getVecs(clean_test, model, num_features)

<ipython-input-23-1562c0dd135e>:10: RuntimeWarning: invalid value encountered in divide
  vec = np.divide(vec,noOfWords)


In [24]:
training_vectors.shape

(9083, 300)

In [25]:
training_vectors = np.array(training_vectors)
testing_vectors = np.array(testing_vectors)

# Reshaping train and test vectors to 3 dimensions. (1 represnts one timestep)
training_vectors = np.reshape(training_vectors, (training_vectors.shape[0], 1, training_vectors.shape[1]))
testing_vectors = np.reshape(testing_vectors, (testing_vectors.shape[0], 1, testing_vectors.shape[1]))
lstm_model = get_model()

/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ lstm (LSTM)                          │ (None, 1, 300)              │         721,200 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_1 (LSTM)                        │ (None, 64)                  │          93,440 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 64)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 1)                   │              65 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 814,705 (3.11 MB)

 Trainable params: 814,705 (3.11 MB)

 Non-trainable params: 0 (0.00 B)

In [26]:
training_vectors.shape

(9083, 1, 300)

In [27]:
lstm_model.fit(training_vectors, y_train, batch_size=64, epochs=150)

Epoch 1/150
142/142 ━━━━━━━━━━━━━━━━━━━━ 8s 18ms/step - loss: 20.7555 - mae: 3.7699
Epoch 2/150
142/142 ━━━━━━━━━━━━━━━━━━━━ 6s 24ms/step - loss: 5.6753 - mae: 1.8955
Epoch 3/150
142/142 ━━━━━━━━━━━━━━━━━━━━ 5s 33ms/step - loss: 5.3489 - mae: 1.8338
Epoch 4/150
142/142 ━━━━━━━━━━━━━━━━━━━━ 5s 30ms/step - loss: 5.1996 - mae: 1.8061
Epoch 5/150
142/142 ━━━━━━━━━━━━━━━━━━━━ 3s 21ms/step - loss: 5.1515 - mae: 1.7969
Epoch 6/150
142/142 ━━━━━━━━━━━━━━━━━━━━ 5s 17ms/step - loss: 5.2462 - mae: 1.8157
Epoch 7/150
142/142 ━━━━━━━━━━━━━━━━━━━━ 4s 28ms/step - loss: 5.1379 - mae: 1.7873
Epoch 8/150
142/142 ━━━━━━━━━━━━━━━━━━━━ 4s 20ms/step - loss: 5.1026 - mae: 1.7936
Epoch 9/150
142/142 ━━━━━━━━━━━━━━━━━━━━ 5s 17ms/step - loss: 5.1259 - mae: 1.7807
Epoch 10/150
142/142 ━━━━━━━━━━━━━━━━━━━━ 3s 19ms/step - loss: 5.1408 - mae: 1.7826
Epoch 11/150
142/142 ━━━━━━━━━━━━━━━━━━━━ 6s 23ms/step - loss: 4.9967 - mae: 1.7565
Epoch 12/150
142/142 ━━━━━━━━━━━━━━━━━━━━ 4s 18ms/step - loss: 4.9696 - mae: 1.7557


In [28]:
lstm_model.save('final_lstm.h5')
y_pred = lstm_model.predict(testing_vectors)
y_pred = np.around(y_pred)
y_pred

122/122 ━━━━━━━━━━━━━━━━━━━━ 2s 11ms/step


array([[3.],
       [5.],
       [6.],
       ...,
       [7.],
       [7.],
       [9.]], dtype=float32)